In [1]:
import pandas as pd
import numpy as np
import random
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, plot_confusion_matrix, make_scorer
from sklearn.model_selection import train_test_split, KFold, cross_val_score
from matplotlib import pyplot as plt

# Read Data

In [2]:
# GALAXY DATASET
def DatasetGALAXY():
    df = pd.read_csv('../../Data/galaxymorphology/dataset1_sydney.csv')
    df.head()
    df['target'].value_counts()
    mapping = {
        'elliptical': 0,
        'spiral':1
    }
    df['target'] = df['target'].map(mapping)
    x = df.sample(df.shape[0])
    df = x.iloc[:, :5]
    df['target'] = x.iloc[:,-1]
    return df

df = DatasetGALAXY()
header = df.columns

In [110]:
#CAR DATASET
def DatasetCAR():
    df = pd.read_csv('../../Data/dataset1-car/car_dataset.csv')
    
    #     df['buying'].replace(['vhigh', 'high', 'med', 'low'],[3,2,1,0], inplace = True)
    #     df['maintenance'].replace(['vhigh', 'high', 'med', 'low'],[3,2,1,0], inplace = True)
    #     df['doors'].replace(['2', '3', '4', '5more'],[2,3,4,5], inplace = True)
    #     df['persons'].replace(['2', '4', 'more'],[2,4,5], inplace = True)
    #     df['lug_boot'].replace(['small', 'med', 'big'],[1,2,3], inplace = True)
    #     df['safety'].replace(['high', 'med', 'low'],[3,2,1], inplace = True)
    #     df['target'].replace(['unacc', 'acc', 'good', 'vgood'],[0,1,2,3], inplace = True)
    #     df= df[df.target !=3]
    #     df= df[df.target !=2]
    print(df['target'].nunique())
    df = df.sample(df.shape[0])
    return df

df = DatasetCAR()
header = df.columns

4


In [104]:
#IRIS DATASET
def DatasetIRIS():
    df = pd.read_csv('../../Data/iris/iris.data')
    df['target'].replace(['Iris-setosa', 'Iris-versicolor', 'Iris-virginica'],[0,1,2], inplace = True)
    df = df.sample(df.shape[0])
    return df
df = DatasetIRIS()
header = df.columns

In [107]:
#TITANIC DATASET
def DatasetTITANIC():
    df = pd.read_csv('../../Data/Titanic/train.csv')
    df.drop(columns = [ 'Cabin','Embarked', 'Ticket','Name','PassengerId'], inplace=True)
    df['Sex'].replace(['male', 'female'],[0,1], inplace = True)
    df =df.dropna()
    
    return df
df = DatasetTITANIC()
header = df.columns

In [3]:
df

,u-g,g-r,r-i,i-z,ecc,target
459,1.31450,0.58474,0.33902,0.18575,0.910293,1
498,1.70491,0.84471,0.40622,0.32586,0.648506,1
464,1.02912,0.36241,0.24938,0.20448,0.684930,1
57,1.97454,0.96352,0.43336,0.34715,0.721961,0
495,1.20532,0.48992,0.27027,0.10746,0.600674,1
...,...,...,...,...,...,...
270,1.35120,0.58938,0.32680,0.21697,0.819921,1
156,1.96622,0.97002,0.43299,0.32953,0.827366,0
62,1.90571,0.86412,0.38059,0.31209,0.865797,0
143,1.98308,0.89501,0.45316,0.30961,0.876977,0


# Random Forest

In [62]:
#Counts the number of each type of example in a dataset
def count_targets(rows):
    count = {}  # a dictionary of label -> count.
    for row in rows:
        label = row[-1]
        if label not in count:
            count[label] = 0
        count[label] += 1
    return count

In [63]:
def is_numeric(value):
    return isinstance(value, int) or isinstance(value, float)

In [64]:
# the splitting question
class Question:

    def __init__(self, column, value):
        self.column = column #1
        self.value = value #3
    
    #Check if a row answers the question
    def check_answer(self, example): 
        value = example[self.column]
        if is_numeric(value):
            return value >= self.value
        else:
            return value == self.value
    
    # prints the question
    def __repr__(self):
        condition = "=="
        if is_numeric(self.value):
            condition = ">="
        return f" {header[self.column]} {condition} {str(self.value)}?"


In [65]:
# splits the dataset according to the question
# true rows satisfy the question 
def split(rows, question):
    correct_rows, wrong_rows = [], []
    for row in rows:
        if question.check_answer(row):
            correct_rows.append(row)
        else:
            wrong_rows.append(row)
    return correct_rows, wrong_rows

In [66]:
#Calculate the Gini Impurity for a list of rows.
# gini equation was taken from here
#https://en.wikipedia.org/wiki/Decision_tree_learning#Gini_impurity

def gini_impurity(rows):
    count = count_targets(rows)
    impurity = 1
    for label in count:
        label_probability = count[label] / float(len(rows))
        impurity -= label_probability**2
    return impurity

In [67]:
# just like in the quiz
def info_gain(left, right, current_uncertainty):
    probability = float(len(left)) / (len(left) + len(right))
    return current_uncertainty - probability * gini_impurity(left) - (1 - probability) * gini_impurity(right)

In [68]:
def best_splitting_feature(rows, random_subspace):

    highest_info_gain = 0  # keep track IG
    highest_info_gain_question = None 
    current_uncertainty = gini_impurity(rows)
    column_indices = list(range(len(rows[0]) - 1))  # number of columns
    
    if random_subspace and random_subspace <= len(column_indices):
        column_indices = random.sample(population = column_indices, k = random_subspace)
    
    for col in column_indices:  #each feature

        values = set([row[col] for row in rows])  # unique values in the column
        
        for val in values:  # for each value

            question = Question(col, val)

            correct_rows, wrong_rows = split(rows, question)

            # Skip split if it doesn't divide the dataset.
            if len(correct_rows) == 0 or len(wrong_rows) == 0:
                continue

            # Calculate the information gain from this split
            gain = info_gain(correct_rows, wrong_rows, current_uncertainty)

            if gain >= highest_info_gain:
                highest_info_gain, highest_info_gain_question = gain, question

    return highest_info_gain, highest_info_gain_question

In [69]:
class Leaf_node:
    def __init__(self, rows):
        self.predictions = count_targets(rows)


In [70]:
class Decision_Node:

    def __init__(self, question, right_branch, wrong_branch):
        self.question = question
        self.right_branch = right_branch
        self.wrong_branch = wrong_branch

In [71]:
def build_tree_function(rows, random_subspace):

    info_gain, question = best_splitting_feature(rows, random_subspace)    
    if info_gain == 0:
        return Leaf_node(rows)

    
    correct_rows, wrong_rows = split(rows, question)
    right_branch = build_tree_function(correct_rows, random_subspace)
    left_branch = build_tree_function(wrong_rows, random_subspace)

    return Decision_Node(question, right_branch, left_branch)

In [72]:
def classify(row, node):
    
    if isinstance(node, Leaf_node):
        return node.predictions

    # split
    if node.question.check_answer(row):
        return classify(row, node.right_branch)
    else:
        return classify(row, node.wrong_branch)

In [73]:
def print_leaf(counts):
    total = sum(counts.values()) * 1.0
    probs = {}
    for lbl in counts.keys():
        probs[lbl] = str(int(counts[lbl] / total * 100)) + "%"
        
    return probs

In [74]:
def bootstrapping(dataset, num_bootstrap):
    """
    Idea is to create random sample for each decision tree that we create
    """
    bootstrap_indices = np.random.randint(low=0, high=len(dataset), size= num_bootstrap)
    df_bootstrapped = dataset.iloc[bootstrap_indices]
    return df_bootstrapped

In [75]:
def random_forest(dataset, n_features,n_decision_trees = 50, num_bootstrap=50):
    forest = []
    for i in range(n_decision_trees):
        bootstrapped_data = bootstrapping(dataset, num_bootstrap)
        bootstrapped_data = bootstrapped_data.to_numpy()
        tree = build_tree_function(bootstrapped_data, random_subspace=n_features)
        forest.append(tree)
    return forest

In [76]:
def random_forest_predictions(test_data, forest):
    test_predictions = {}
    test_data = test_data.to_numpy()
    
    for i in range(len(forest)):
        column_name = f"tree_{i}"
        predicted= []
        actual = []
        
        for row in test_data:
            most_probable = print_leaf(classify(row, forest[i]))
            predicted += [next(iter(most_probable))]
            actual += [row[-1]]
        test_predictions[column_name] = predicted

        
    test_predictions = pd.DataFrame(test_predictions)
    print(test_predictions)
    random_forest_predictions = test_predictions.mode(axis=1)[0]
    
    return random_forest_predictions, actual

In [77]:
def get_overall_cm(df):
    classes = df['target'].nunique()
    if classes < 2:
        classes = 2
    overall_cm = []
    for i in range(classes):
        overall_cm.append( [0] * classes)
    return overall_cm

In [78]:
from sklearn.metrics import confusion_matrix

def Kfold(df,n_folds, test_size, overall_cm):
    # Iterate through all folds
    overall_predicted = []
    overall_actual = []
    for i in range(n_folds):
        print("\n\n\n\nTHIS IS FOLD:", i)
        #split test and train
        test = df.iloc[test_size * i : test_size*i+test_size]
        
        train = df.iloc[test_size:]

        forest = random_forest(train, n_features = 10, n_decision_trees = 10, num_bootstrap = 500)

        predicted, actual = random_forest_predictions(test, forest)
        for p,a in zip(predicted,actual):
            overall_predicted.append(p)
            overall_actual.append(a)
        # generate and append cm 
        overall_cm = overall_cm + confusion_matrix(actual, predicted)
    
    print(classification_report(overall_predicted, overall_actual))
    return overall_cm


In [79]:

if __name__ == "__main__":
    # KFOLD take in n folds and the size of dataset
    
    # DATASET1 SYDNEY
    n_folds = 2
    test_size = int(df.shape[0]/n_folds)

    # get overall cm returns a Confusion matrix with 0's shape depends on (classes)
    rfcm = Kfold(df,n_folds, test_size, get_overall_cm(df))
    print(f"\n\n\n Confusion Matrix\n{rfcm}")





THIS IS FOLD: 0
     tree_0  tree_1  tree_2  tree_3  tree_4  tree_5  tree_6  tree_7  tree_8  \
0       1.0     1.0     1.0     1.0     1.0     1.0     1.0     1.0     1.0   
1       1.0     1.0     1.0     1.0     1.0     1.0     1.0     1.0     1.0   
2       1.0     1.0     1.0     1.0     1.0     1.0     1.0     1.0     1.0   
3       0.0     0.0     0.0     0.0     0.0     0.0     0.0     0.0     0.0   
4       1.0     1.0     1.0     1.0     1.0     1.0     1.0     1.0     1.0   
..      ...     ...     ...     ...     ...     ...     ...     ...     ...   
255     1.0     1.0     1.0     1.0     1.0     1.0     1.0     1.0     1.0   
256     1.0     1.0     1.0     1.0     1.0     1.0     1.0     1.0     1.0   
257     1.0     1.0     1.0     1.0     1.0     1.0     1.0     1.0     1.0   
258     0.0     0.0     0.0     0.0     0.0     0.0     1.0     0.0     0.0   
259     0.0     0.0     0.0     0.0     0.0     0.0     0.0     0.0     0.0   

     tree_9  
0       1.0  
1  